# HMM Viterbi Algorithm implementation on Amazon product review data set

We have implemented Viterbi algorithm for text predection on  Amazon reviews dataset.
Hidden/Transition states are = Words in the review(Considered unique word set as states)

Transition probability (one state (word) to other word)

Emission probability :Every review is our observation. According, the thoeryof HMM We have Sentences/reviews provided so that we can uses as observations and calculate the probability of different states/words from this observations.

Initial probability : Occurace of a word/state in total no.of states/word.
Transition probability : Probabilty(word1|word2)
Emission Probability : Probability (word/no.of words in the passage)*normallization value
Viterbi Porbability = (viterbi_probability of last entry)*(Transion probability of state)*(Emission probability for sate to observation)

Viterbi sequence formation  with max probabilites at each step 
Backtrac sequece : a dictionary of max probability as values and predicted state)

Text Prediction is done using HMM Viterbi algorithm. Time span is noted down.


In [277]:
#Required libraires for the project
import os
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import re
df = pd.read_csv('Reviews.csv',sep=',',engine='python')

In [278]:
#Main data file
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
Id                        568454 non-null int64
ProductId                 568454 non-null object
UserId                    568454 non-null object
ProfileName               568438 non-null object
HelpfulnessNumerator      568454 non-null int64
HelpfulnessDenominator    568454 non-null int64
Score                     568454 non-null int64
Time                      568454 non-null int64
Summary                   568427 non-null object
Text                      568454 non-null object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [279]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [280]:
#Created a new data set from base one
df1= pd.DataFrame(df['Text'][0:8000],columns=['Text'])

In [281]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 1 columns):
Text    8000 non-null object
dtypes: object(1)
memory usage: 62.6+ KB


In [282]:
#Used Natual language processing tool kit to word tokenize the data
from nltk import word_tokenize
df1['Text'].dropna(inplace=True)
df1['tokenized_text'] =df1['Text'].apply(word_tokenize)

In [283]:
df1['tokenized_text'].head()

0    [I, have, bought, several, of, the, Vitality, ...
1    [Product, arrived, labeled, as, Jumbo, Salted,...
2    [This, is, a, confection, that, has, been, aro...
3    [If, you, are, looking, for, the, secret, ingr...
4    [Great, taffy, at, a, great, price, ., There, ...
Name: tokenized_text, dtype: object

In [284]:
#Tokenzied output sample
df1['tokenized_text'][0]

['I',
 'have',
 'bought',
 'several',
 'of',
 'the',
 'Vitality',
 'canned',
 'dog',
 'food',
 'products',
 'and',
 'have',
 'found',
 'them',
 'all',
 'to',
 'be',
 'of',
 'good',
 'quality',
 '.',
 'The',
 'product',
 'looks',
 'more',
 'like',
 'a',
 'stew',
 'than',
 'a',
 'processed',
 'meat',
 'and',
 'it',
 'smells',
 'better',
 '.',
 'My',
 'Labrador',
 'is',
 'finicky',
 'and',
 'she',
 'appreciates',
 'this',
 'product',
 'better',
 'than',
 'most',
 '.']

In [285]:
#Check stop words and special characters
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')
special=['!','@','#','$','%','^','&','*','(',')',':','<','>',',','.',';','{','}','|','_','-','+','=','`','~',"--","'",'"','[',']']

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prashanthimallijula/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [286]:
#Remove speacial characters
for item  in df1['tokenized_text']:
    item_new=[]
    for word in item:
        word=str(word)
        if (word in special):
            item.remove(word)

In [287]:
df1.head()

,Text,tokenized_text
0,I have bought several of the Vitality canned d...,"[I, have, bought, several, of, the, Vitality, ..."
1,Product arrived labeled as Jumbo Salted Peanut...,"[Product, arrived, labeled, as, Jumbo, Salted,..."
2,This is a confection that has been around a fe...,"[This, is, a, confection, that, has, been, aro..."
3,If you are looking for the secret ingredient i...,"[If, you, are, looking, for, the, secret, ingr..."
4,Great taffy at a great price. There was a wid...,"[Great, taffy, at, a, great, price, There, was..."


In [288]:
#Create trasition states
trans_states=[]
for item  in df1['tokenized_text']:
        temp_list = []
        for i in range(0,len(item)) :
            #if i+1 == len(item):
                    #temp_list.append((item[i],"end"))
            if i+1 < len(item):
                    temp_list.append((item[i],item[i+1]))
        trans_states.append(temp_list)
df1['trans_states'] =trans_states

In [289]:
df1.head()

,Text,tokenized_text,trans_states
0,I have bought several of the Vitality canned d...,"[I, have, bought, several, of, the, Vitality, ...","[(I, have), (have, bought), (bought, several),..."
1,Product arrived labeled as Jumbo Salted Peanut...,"[Product, arrived, labeled, as, Jumbo, Salted,...","[(Product, arrived), (arrived, labeled), (labe..."
2,This is a confection that has been around a fe...,"[This, is, a, confection, that, has, been, aro...","[(This, is), (is, a), (a, confection), (confec..."
3,If you are looking for the secret ingredient i...,"[If, you, are, looking, for, the, secret, ingr...","[(If, you), (you, are), (are, looking), (looki..."
4,Great taffy at a great price. There was a wid...,"[Great, taffy, at, a, great, price, There, was...","[(Great, taffy), (taffy, at), (at, a), (a, gre..."


In [290]:
#Create a dictionary of transtion states and corresponding probabilites
trans_states={}
dict_trans_states=[]
for item  in df1['trans_states']:
    c=0
    c1=0
    trans_states={}
    for i in item :
        c=0
        c1=0
        for j in item:
            if i == j:
                c= c+1
            if i[0]==j[0]:
                c1=c1+1
        trans_states[i]=c/c1
    dict_trans_states.append(trans_states)
df1['dict_trans_states'] =dict_trans_states

In [291]:
#Convert it as data frame
trans_dict={}
for item in df1['dict_trans_states']:
    for key,value in item.items():
        trans_dict[key]=value

trans_df = pd.DataFrame(trans_dict.items())       
trans_df.columns=['trans_entry','trans_Probability']
trans_df['trans_entry1']=trans_df['trans_entry']

In [292]:
trans_df.set_index('trans_entry', inplace=True)

In [293]:
#Get uniques states in the data that is unique words in our case
unique_states=[]
temp_list=[]
for item in df1['tokenized_text']:
    for word in item:
        temp_list.append(word)
temp_list =set(temp_list)
unique_states=temp_list

In [294]:
#Make sentences Lower case
df1['text_lowercase']= df1['Text'].str.lower()

In [295]:
#Sample Tansition probability output
df1['dict_trans_states'][0]

{('I', 'have'): 1.0,
 ('Labrador', 'is'): 1.0,
 ('My', 'Labrador'): 1.0,
 ('The', 'product'): 1.0,
 ('Vitality', 'canned'): 1.0,
 ('a', 'processed'): 0.5,
 ('a', 'stew'): 0.5,
 ('all', 'to'): 1.0,
 ('and', 'have'): 0.3333333333333333,
 ('and', 'it'): 0.3333333333333333,
 ('and', 'she'): 0.3333333333333333,
 ('appreciates', 'this'): 1.0,
 ('be', 'of'): 1.0,
 ('better', 'My'): 0.5,
 ('better', 'than'): 0.5,
 ('bought', 'several'): 1.0,
 ('canned', 'dog'): 1.0,
 ('dog', 'food'): 1.0,
 ('finicky', 'and'): 1.0,
 ('food', 'products'): 1.0,
 ('found', 'them'): 1.0,
 ('good', 'quality'): 1.0,
 ('have', 'bought'): 0.5,
 ('have', 'found'): 0.5,
 ('is', 'finicky'): 1.0,
 ('it', 'smells'): 1.0,
 ('like', 'a'): 1.0,
 ('looks', 'more'): 1.0,
 ('meat', 'and'): 1.0,
 ('more', 'like'): 1.0,
 ('of', 'good'): 0.5,
 ('of', 'the'): 0.5,
 ('processed', 'meat'): 1.0,
 ('product', 'better'): 0.5,
 ('product', 'looks'): 0.5,
 ('products', 'and'): 1.0,
 ('quality', 'The'): 1.0,
 ('several', 'of'): 1.0,
 ('she',

In [296]:
#Calculation of Term frequency and used as emiziion probability and 
#TDIDF will be used in many case to check the importance of the word 
def tf(item):
    tf_numerator= item[1].count(item[0])
    tf_denominator = len(item[1].split())
    TF=tf_numerator/float(tf_denominator)
    return (TF)
emission_dict={}
temp_l=[]
i=0
for word in unique_states:
        for item in df1['text_lowercase']:
            if word in item: 
                temp_l = [word,item]
                key=tuple(temp_l)
                TF=tf(key)
                emission_dict[key] = float(TF)

In [297]:
#sample Emission probability dictionary values Different observations and one state
i=0
for key,value in emission_dict.items():
    if i<=5:
        print(key)
        print(" ")
        print(emission_dict[key])
        i=i+1

('week.', "one of my boys needed to lose some weight and the other didn't.  i put this food on the floor for the chubby guy, and the protein-rich, no by-product food up higher where only my skinny boy can jump.  the higher food sits going stale.  they both really go for this food.  and my chubby boy has been losing about an ounce a week.")
 
0.015625
('week.', 'use frequently as we like to do asian dishes at least once a week.  love this product.  fast shipping, as usual.  would buy again.')
 
0.041666666666666664
('week.', "i am a preschool teacher and do a g is for gingerbread unit every january. i was on vacation during christmas and when i got back not a gingerbread kit was to be found. which is odd. i needed one for my preschool class and needed it in less then a week. i looked everywhere and could not find ont. so i had one overnighted from amazon on nye and it came yesterday. i pick this one because of the rudolph theme. the gingerbread itself came broken. like unable to use wou

In [298]:
#Preparing inital probability for all states
initial_prob={}
for word1 in unique_states:
    c=0
    for word2 in unique_states:
        if word1==word2:
              c=c+1
    initial_prob[word1]=c/len(unique_states)

In [299]:
#Converting it to data frame for better usage
init_df = pd.DataFrame(initial_prob.items())
init_df.columns=['init_entry','init_Probability']
init_df.set_index('init_entry', inplace=True)

In [300]:
#Sample initial probabilites
init_df.head()

,init_Probability
init_entry,
week.,0.000038
infant,0.000038
anticipating,0.000038
foils,0.000038
murky,0.000038


In [301]:
# seed the pseudo random number generator .  A randome number picke to get a random paragraph to work on 
from random import randint
# seed random number generator
rand_int = randint(0, 7999)
print(rand_int)
print(df1['text_lowercase'][rand_int])

6741
i usually cringe a little before i do a natural juice/carbonated beverage.  there is frequently something of an aftertaste that is unpleasant.<br /><br />this is refreshing, light, very bubbly and leaves a bright taste after with no cringe effects.<br /><br />i definitely would buy this drink.


In [302]:
random_list=[]
random_list= df1['tokenized_text'][rand_int]
text=df1['text_lowercase'][rand_int]
print(text)

i usually cringe a little before i do a natural juice/carbonated beverage.  there is frequently something of an aftertaste that is unpleasant.<br /><br />this is refreshing, light, very bubbly and leaves a bright taste after with no cringe effects.<br /><br />i definitely would buy this drink.


In [303]:
#Somethods to extract Emission probability, Transition probability, Initial probability and states list for the paragraph
def get_obs_text(cur_state):
    list1=[]
    for key1,value1 in emission_dict.items():
        if  key1[0] == cur_state:
            list1.append(float(value1))
        else:
            list1.append(0)
    considered_obs=max(list1)
    for key2,value2 in emission_dict.items():
          if value2 == considered_obs:
                return(key)

def get_required_states(text):  
    states_list=[]
    obs_list=[]
    for key,value in emission_dict.items():
            if (len(key[0])>3) and (key[1] == text):
                states_list.append(key[0])

    states_list = set(states_list)
    states_list=list(states_list)
    for i in states_list:
        for item in df1['text_lowercase']:
            if i in item:
                obs_list.append(item)
    obs_list=set(obs_list)
    obs_list=list(obs_list)
    return(states_list,obs_list)


def get_init_prob(word):
    if init_df.loc[word,'init_Probability'] !=0:
        init_prob= init_df.loc[word,'init_Probability'] #Get the initial probability of the word
    else:
        init_prob=0 
    return init_prob
def get_em_prob(temp_tuple):
    if emission_dict[temp_tuple] !=0:
         v_prob=emission_dict[temp_tuple]
    else:
        v_prob=0
    return v_prob

def trans_prob(st1,st2):
    word=[st1,st2]
    word=tuple(word)
    if trans_dict[word] !=0:
        trans_prob= trans_dict[word]
    else:
        trans_prob=1 
    return(trans_prob)


def get_req_trans_states(state):
    req_trans_states=[]
    tr=[]
    for i in trans_df['trans_entry1']:
        if (i[0] == state) :
            req_trans_states.append(i[1])
            tr.append(i)
    return(req_trans_states,tr)

In [304]:
#Starting HMM VITERBI ALGORITHM FROM HERE ,FUNCTIONS REQUIRED ARE DEFINE ABOVE
def viterbi_algo(text,states_list,obs_list,trans_state_list,t,state,viterbi_prob_sequence,backtrack_states):
    #Initial probabilites of inital states considering corresponding observation.
    while (t<10):
        temp_v_dict={}
        #print(t)
        if t==0:
            v_init_prob=get_init_prob(state)
            temp_list=[state,text]
            temp_tuple= tuple(temp_list)
            v_init_emision= get_em_prob(temp_tuple)
            v_entry= v_init_prob*v_init_emision
            viterbi_prob_sequence.append(v_entry)
            backtrack_states[state]= v_entry
        for st in trans_state_list:
                if (len(st)>=3) and (st.isalpha()==True):
                    #print("st",st)
                    text1=get_obs_text(st.lower())
                    if text1 != None:
                        v_emision_prob=get_em_prob(text1)
                        v_trans_prob= trans_prob(state,st)
                        v_entry= viterbi_prob_sequence[-1]*v_emision_prob*v_trans_prob
                        temp_v_dict[st]=v_entry
        max_prob=max(temp_v_dict.values())
        viterbi_prob_sequence.append(max_prob)
        i=0
        for k,v in temp_v_dict.items():
            if (v==max_prob) and (i==0):
                backtrack_states[k]= max_prob
                #print(k,v)
                new_state=k
                i=i+1
    
        #calculate next level list
        new_text=get_obs_text(new_state)
        states_list,obs_list = get_required_states(new_text)
        trans_state_list,tr = get_req_trans_states(new_state)
        t=t+1
        #print("here:", t)
        if t<10:
            viterbi_algo(new_text,states_list,obs_list,trans_state_list,t,new_state,viterbi_prob_sequence,backtrack_states)
        return (viterbi_prob_sequence,backtrack_states)

# Viterbi sequence and backtrack calculation
viterbi_prob_sequence=[] 
backtrack_state={}


#Extracting required states and observations for the picked random corpus
states_list,obs_list = get_required_states(text)
state = states_list[-1]
trans_state_list,tr = get_req_trans_states(state)
#Limited no.of iterations to predict only to words after the end.
t=0
viterbi_prob_sequence,backtrack_states= viterbi_algo(text,states_list,obs_list,trans_state_list,t,state,viterbi_prob_sequence,backtrack_states)
value_list=[]
#Print the output
print('Viterbi probability seqence:')
print("")
print(viterbi_prob_sequence)
print("")
print("Track path:")
print("")
for key,value in backtrack_states.items():
        print (key ,":", value)
        value_list.append(key)
print("")
print("Input text(Random picked)")
print("")
print(text)
print("  ")
print("continued with Predicted text..." )
print("Predicted text:" )
print("  ")
print (value_list)
print("  ")
print("combined with input")
print(text)
print (value_list)
print("  ")
print("Program Complted")

Viterbi probability seqence:

[1.6325571068475976e-06, 2.0744054724874177e-09, 2.635839228065334e-12, 3.3492239238441347e-15, 4.255684782521137e-18, 5.407477487320378e-21, 6.871000619212678e-24, 8.730623404336314e-27, 1.1093549433718314e-29, 1.409599673915923e-32, 1.791105049448441e-35]

Track path:

appetizing : 5.867002102733554e-07
Once : 7.454894666751657e-10
you : 9.472547225859793e-13
buying : 1.2036273476314858e-15
this : 1.5293867187185336e-18
product : 1.943312222005761e-21
looks : 2.4692658475295565e-24
more : 3.137567785933363e-27
than : 3.9867443277425194e-30
most : 5.065748828135349e-33
have : 6.436783771455335e-36
like : 7.509762691498949e-07
cool : 5.074163980742533e-07
place : 6.447476468541974e-10
because : 8.192473276419282e-13
they : 1.0409750033569608e-15
were : 1.3227128378106236e-18
stuck : 1.6807024622752525e-21
together : 2.1355812735390755e-24
which : 2.7135721391856105e-27
was : 3.4479950942638e-30
made : 4.381188175684625e-33
some : 5.566948126664073e-36
caff